## Data extraction and visualizing

- Merging data from other files
- Read the data from Archigos excel document/dta data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Alternative to set svg for newer versions
%matplotlib inline


# Showing all the details of results
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# %matplotlib qt

In [ ]:
# Read the original dta(Stata) data download from Archigos website.
# load the data
datafile = './../Data/Archigos_4.1_stata14.dta'
data = pd.read_stata(datafile)

# Startdate is better than eindate, enddate is better than eoutdate
# Change the data type of startdate and enddate to datatime
data['startdate'] = data['startdate'].astype('datetime64[ns]')
data['enddate'] = data['enddate'].astype('datetime64[ns]')
data['yrbegin'] = pd.DatetimeIndex(data['startdate']).year
data['yrend'] = pd.DatetimeIndex(data['enddate']).year

# Import the data about countries grouped by three categories: Autocracy, Anocracy, Democracy
data_deau = pd.read_excel('./../Data/DeorAu_group.xlsx')

# load another excel file mpd2020 with gdppc and pop
data_mpd2020 = pd.read_excel('./../Data/mpd2020.xlsx', 'Full data')

# Combine data_deau with data_leader with key id=countries
# Rename the Country-id to idacr in data_deau
data_deau = data_deau.rename({'Country-id': 'idacr'}, axis='columns')
data_leader_deau = pd.merge(data, data_deau, how ='left', on ='idacr')

# Rename the countrycode to idacr in data_mpd2020
# Rename the year in gdppc to yrend to merge data
data_mpd2020 = data_mpd2020.rename({'countrycode': 'idacr', 'year': 'yrend'}, axis='columns')

# Change the float to int
data_leader_deau['yrend'] = data_leader_deau['yrend'].fillna(0.0).astype(int)
data_leader_deau['yrbegin'] = data_leader_deau['yrbegin'].fillna(0.0).astype(int)

# Combine data_leaders_deau with data_mpd2020 with key idacr and year
data_leader_deau_gdp = pd.merge(data_leader_deau, data_mpd2020, how ='left', on =['idacr', 'yrend'])

# Rename the gdppc to end_gdppc in data_leaders_deau_gdp
data_leader_deau_gdp = data_leader_deau_gdp.rename({'gdppc': 'end_gdppc'}, axis='columns')

data_gdp = pd.read_excel('./../Data/data_gdp.xlsx')
data_gdp = data_gdp.rename({'year': 'yrbegin'}, axis='columns')
data_all = pd.merge(data_leader_deau_gdp, data_gdp, how ='left', on =['idacr', 'yrbegin'])

# Rename the gdppc to begin_gdppc in data_all
data_all = data_all.rename({'gdppc': 'begin_gdppc'}, axis='columns')

# Calculate the leaders' age, when they first into power
# See the characteristics of yrborn, there is something wrong in the data. 
# Because I change the data type of yrborn, from int to datetime.

# age= yrbegin- yrborn
data_all['age'] = data_all['yrbegin'] - data_all['yrborn']

# See the distribution of age
# data_all.groupby(['age']).mean()

## Calculate their tenure
 
- Calculate the tenure
- Check the data about our data about tenure 
- Delete tenure < 1
- Group the data by leaders and sum their tenure together ？？

In [ ]:
# tenure= enddate or eoutdate - startdate  or  eindate (Unit: years)
# if enddate is null, it has two possibilities: 
# 1. leader is still in office, use the date now; 2. not available.  

import datetime

data_all['startdate'] = pd.to_datetime(data_all['startdate']).dt.date
data_all['enddate'] = pd.to_datetime(data_all['enddate']).dt.date

data_all['enddate'] = np.where(data_all['enddate'].isnull(),
                                datetime.datetime.now().date(),
                                data_all['enddate'])

data_all['tenure'] = (data_all['enddate'] - data_all['startdate'])/np.timedelta64(1, 'Y')

# data_all.groupby(['tenure']).mean()

data_all.drop(data_all[data_all['tenure'] < 1].index , inplace=True)

# Add tenure to the excel grouped by leaders
# Some leaders may have more than one term
# Tenure: merge the duplicates and sum the tenure.
# data_all=data_all.groupby(['leader']).sum()

# Check if there are any abonormal data_all.
# data_all.groupby(['tenure']).mean()

In [ ]:
# calulate the growth rate
data_all['growth_rate'] = (data_all['end_gdppc'] / data_all['begin_gdppc'] - 1) / data_all['tenure']

# data_all.describe()

data_all.dropna(subset = ['growth_rate'], inplace=True)

## Data cleaning

### Delete the irrelevant data with irrelevant columns

In [ ]:
# 'country_x','country_y'
data_all.drop(columns=['obsid',
                   'leadid',
                   'idacr',
                   'startdate',
                   'eindate',
                   'enddate',
                   'eoutdate',
                   'borndate',
                   'ebirthdate',
                   'deathdate',
                   'edeathdate',
                   'dbpediauri',
                  'numexitcode',
                  'numposttenurefate',
                  'ftcur',
                  'Country',
                  'Polity datasets IV number[11][12]',
                  'Polity datasets IV category'], inplace=True)

# Deal with family ties and gender 
data_all['fties_range'] = np.where(data_all['fties'].isnull(), 0.5, 1)
data_all['gender'] = np.where(data_all['gender']=='M', 0.5, 1)


data_all.drop(columns=['fties',
                       'Democracy',
                       'Autocracy'], inplace=True)
                       
# Delete the columns that more than 30% is null 
# Delete the column >= 30% is null, we can see the remaining column names.
null_sum=data_all.isnull().sum()
data_all.columns[null_sum<len(data_all)*0.3] # columns will keep

# Drop the columns that at least 30% values are null
data_all.drop(columns=data_all.columns[null_sum > len(data_all)*0.3], inplace=True)

# Check if there are any abnormal values, we need to pay attention to the negative numbers. 
# Some leaders are still alive, so the yrdied is negative number, etc.
# print(data.describe())

In [ ]:
data_all.dropna(subset = ['leader'], inplace=True)

In [ ]:
# Delete if yrbegin > yrend, age < 0
# print(data_all[data_all['yrbegin'] > data_all['yrend']]) #age < 0
data_all.drop(data_all[data_all['yrbegin'] > data_all['yrend']].index, inplace=True)

# Delete the data_all yrbegin < 0, yrend < 0, yrborn<0, yrdied<0
data_all.drop(data_all[data_all['yrborn']<0].index , inplace=True) 
# data_all.drop(data_all[data_all['yrdied']<0].index, inplace=True)
data_all.drop(data_all[data_all['yrbegin']<0].index , inplace=True) 
data_all.drop(data_all[data_all['yrend']<0].index, inplace=True)
# print(data_all.shape)

# Find if yrbegin > yrdied, lifespan < 0 
# print(data_all[data_all['yrbegin'] > data_all['yrdied']]) # begin to come into power after the leader is died.

# Delete the data_all yrbegin > yrdied
# data_all.drop(data_all[data_all['yrbegin'] > data_all['yrdied']].index, inplace=True) 

In [ ]:
data_all.columns

## Visualizing results

In [ ]:
# fig.3
sns.histplot(data_all.entry)

In [ ]:
# fig.4
sns.histplot(data_all.exit, y = data_all.exit, color='blue')

In [ ]:
# See the distribution of age on Fig. 5
sns.histplot(data_all.age,bins=[1,10, 20, 30, 40, 50, 60, 70, 80, 90] ,color = 'orange', shrink=.9, binwidth=2)

In [ ]:
# See the distribution of tenure on fig. 6
sns.histplot(data_all.tenure, color = 'pink', shrink=.9, binwidth=1)

In [ ]:
# Fig.7
# Look at the distribution of the data_all put into the model
ax = sns.histplot(data_all['growth_rate'])

mean = data_all['growth_rate'].mean()
std = data_all['growth_rate'].std()
print(mean)
print(std)